In [12]:
!pip install --upgrade pip setuptools
!pip install selenium
!pip install googletrans==4.0.0-rc1

     ---------------------------------------- 2.1/2.1 MB 2.6 MB/s eta 0:00:00
  Using cached setuptools-67.8.0-py3-none-any.whl (1.1 MB)


ERROR: To modify pip, please run the following command:
C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip setuptools


In [14]:
import os
import sys
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import asyncio
import json
import aiohttp
import openpyxl
import textwrap
#from understat import Understat
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep
import time
import pandas as pd
from googletrans import Translator

In [15]:
#explanation

In [20]:
url='https://www.rest.co.il/kosher-restaurants/israel/kosher/' #creating url for primary pages
header={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.64'}
response=requests.get(url,headers=header)
driver = webdriver.Edge()
page_num=1
resturant_list=[]
infolistArr=[]
info_cnt=0
translator = Translator()
while(page_num<=1):#need 401 pages
    if response.status_code != 200:   # if maximum page exceeded, break out of loop
         break
    html_text=requests.get(url,headers=header)
    mainpage=html_text.text
    mainsoup = BeautifulSoup(mainpage,'lxml') # turn primary page into soup
    rest_page=mainsoup.find_all("div",attrs={"class":"rest-title"})
    
    url_rest_page=[]
    
    for div in rest_page:
        href = div.a['href']
        url_rest_page.append("https://www.rest.co.il"+href)
    for url_rest in url_rest_page:
        html_rest_text=requests.get(url_rest,headers=header)
        restpage=html_rest_text.text
        restsoup = BeautifulSoup(restpage,'lxml')
        rest_name=restsoup.find("div",attrs={"class":"reviews_info_box"})
        rest_name=rest_name.find('h1').text.strip()
        #print(rest_name)
        rest_add=restsoup.find("div", attrs={"class":"h5"}).text.strip()
        try:
            rest_open=restsoup.find("div", attrs={"class":"open_now_dropdown"}).text.strip()
        except AttributeError:
            rest_open='No Info'
        rest_type=restsoup.find("div", attrs={"class":"h6"}).text.strip()
        rest_rank=restsoup.find("div", attrs={"class":"reviews_box"})
        try:
            rest_rank=rest_rank.find("span").text.strip() 
        except AttributeError:
            rest_rank=0
        ratings=restsoup.find_all("div" ,attrs={"class" : "user_progress_widget"})
        for x in ratings:
            name=x.find("span").text.strip()
            rating = x.find("strong").text.strip()
        reviews_num=restsoup.find("div",attrs={"class":"raviews_box_item"})
        
        try:
            reviews_num=reviews_num.find('a').text.strip().replace("חוות דעת",'') 
        except AttributeError:
            reviews_num=0
        driver.maximize_window()
        driver.get(url_rest)
        try:
            wait = WebDriverWait(driver, 10)
            moreinfo = wait.until(EC.visibility_of_element_located((By.XPATH, "//small[@data-event-id='76' and @data-action='open_tags_popup']")))
            moreinfo.click()
            infolist = driver.find_element(By.CLASS_NAME,'pop-scroll-wrap')
            #sleep(5)
            infolistArr.append(infolist.text.strip())
        except:
            infolistArr.append("None")
        print("///////////////////////////////////////////")
        
        temp=[]
        for i,x in enumerate(infolistArr):
            temp.append(infolistArr[i].replace('\n',','))
        new_array=[]
        for i,sub_array in enumerate(temp):
            # Create a temporary list to store the separated parts of each string
            temp_list = []
            sub_array=sub_array.split(',')
            new_array=[]            
            for sub in sub_array:
                new_array.append(f'{sub}')####need to start from here
#                  new_array.append(f'{sub_array[x]} : {sub_array[x]}') 
#             for i in range(info_cnt,len(new_array)):
#                 print(new_array[info_cnt])
#                 new_array[info_cnt].append(new_array[info_cnt])
#                 info_cnt=info_cnt+1
            #new_array.append("$%")
        print(new_array)
        restaurant={'rest name':rest_name,'rest add':rest_add,'rest open':rest_open,'rest type':rest_type,'rest rank':rest_rank,'reviews num':reviews_num}
        for new in new_array:
            new=translator.translate(new, src='he', dest='en')
            restaurant.update({new.text:1})
        #print(restaurant)
        resturant_list.append(restaurant)
        info_cnt=info_cnt+1
    page_num=page_num+1
    url='https://www.rest.co.il/kosher-restaurants/israel/kosher/page-'+ str(page_num) + '/'
    #print(url)   
resturantdf=pd.DataFrame(resturant_list)

///////////////////////////////////////////
['מסעדת שף', 'מסעדה כשרה', 'חדר פרטי', 'מסעדה עם חניה חינם', 'מסעדה עם מרתף יינות', 'נגישות לנכים', 'שירות הזמן שולחן']
///////////////////////////////////////////
['None']
///////////////////////////////////////////
['None']
///////////////////////////////////////////
['None']
///////////////////////////////////////////
['None']
///////////////////////////////////////////
['בר מסעדה', 'מסעדה כפרית', 'עסק שומר שבת', 'מסעדה כשרה', 'מסעדה כשרה למהדרין', 'מסעדה כשרה בד"ץ', 'אפשרות ישיבה בחוץ', 'מסעדה עם חניה מוסדרת', 'נגישות לנכים', 'Take Away', 'שירותי קייטרינג', 'גישה לתחבורה ציבורית', 'הדרכות/סדנאות', 'לוקיישן לצילום', 'אירועים עד 100 איש', 'סוכות', 'אירועי בוטיק', 'אירועים קטנים', 'בר/בת מצווה', 'ברית/בריתה', 'חתונות', 'ימי גיבוש', 'ימי הולדת', 'ימי כיף', 'ימי נישואין', 'ימי עיון', 'ישיבות', 'כנסים', 'מסיבות חברה', 'מסיבות עיתונאים', 'ערבי מחלקה', 'תערוכות']
///////////////////////////////////////////
['None']
///////////////////////////////

In [21]:
# driver = webdriver.Edge()
# url = "https://easy.co.il/list/Restaurants?order=1" 
# cities = ["אשדוד"]#["רמלה","ראשון לציון", "תל אביב", "באר שבע" ,"אילת" ,"נהריה","חיפה","עכו","פתח תקווה","טבריה","ירושלים","הוד השרון","יבנה","הרצליה", "באר טוביה", "ראש פינה", "קרית אתא", "קרית ביאליק", "אלעד", "חד נס", "שוהם", "נתניה", "אשדוד"] 
# driver.get(url)

# for city in cities:
#     changelocation = driver.find_element(By.ID,'changeAddressButton')
#     changelocation.click()
#     sleep(5)
#     search_input = driver.find_element(By.ID,'SearchInputAddress')
#     search_input.send_keys(city + Keys.ENTER)
#     sleep(3)
#     while True:
#         try:
#             load_more_button = driver.find_element(By.CLASS_NAME,'next-page-button')
#             load_more_button.click()
#             sleep(2)
#         except:
#             break

#     Listrestaurants = driver.find_element(By.CLASS_NAME,'list-results')
#     elements = Listrestaurants.find_elements(By.CLASS_NAME, 'biz-name') 
#     for e in elements:
#         print(e.text)

#     driver.refresh()


In [22]:
resturantdf.to_csv(r'C:\tools\project_data_s\Data-Research\Pull_Data.csv',encoding='utf-8-sig',index=False)

In [23]:
resturantdf

,rest name,rest add,rest open,rest type,rest rank,reviews num,Chef Restaurant,Kosher restaurant,private room,Restaurant with free parking,...,Department evenings,Exhibitions,Animal friendly,Deliveries,Chef to the house,An external garden,DOG Friendly,Wi-Fi,Open on Friday,Open in the USSR
0,"דריה - Darya, בהילטון, תל אביב",תל אביב,א'-ה' 18:30-23:00,מסעדת בשרים,0,0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""המסעדה היהודית"", בני ברק","מצדה (כנרת) 4, בני ברק",א'-ה' 12:00-23:00\n\nשעות הפעילות של הגורמה:\n...,ביסטרו,4.5,15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"רג'ינה בתחנה, מתחם התחנה נווה צדק, תל אביב","המרד 3, תל אביב",א'-ה' 17:00-23:30\nו' 11:30-16:00\nש' 19:00-00:00,מסעדת בשרים,4,68,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,גרקו הכשרה - GRECO,"אורי צבי גרינברג 25, צפון תל אביב",א'-ה' 12:00-23:00\nו' 12:00-16:00,מסעדה יוונית,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"רונן בשרים, נתניה","שכטרמן 28, נתניה",א'-ה' 11:00-02:30\nש' חצי שעה אחרי צאת השבת-00:00,מסעדת בשרים,3.5,10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,לחם בשר נמל תל אביב (פתחנו להרצה),"האנגר 14, נמל תל-אביב",א'-ה' 18:00-23:00,מסעדת בשרים,0,0,NaN,1.0,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"דגים ויין DONIS, גבעת שמואל","הערבה 1, גבעת שמואל",א'-ג' 12:00-23:00\nד'-ה' 12:00-00:00\nש' 19:30...,מסעדת דגים,3,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"חצר עין כרם, עין כרם מדרגות הרומאים, ירושלים",ירושלים,א'-ה' 14:00-23:00\nו' 09:30-14:00,מסעדה חלבית,5,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"החשמונאים 7, בני ברק","החשמונאים 7, בני ברק",א'-ד' 14:00-23:00\nה' 13:00-02:00,מסעדת בשרים,5,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"זוקו - zoko by izidor segal, קריית אונו","שד' קק""ל 5, קריית אונו",א'-ה' 18:30-00:00,מסעדת בשרים,5,3,NaN,NaN,NaN,1.0,...,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
